In [ ]:
# importing libraries
import face_recognition
import cv2
import numpy as np
import pandas as pd
import datetime
import time
import os

### Face Training 

In [ ]:
# Path of the directory containing the face images
faces_dir = "face dataset/"

# Initialize lists to store the face encodings and corresponding names
known_face_encodings = []
known_face_names = []
count = 0
# Loop over each file in the faces directory
for filename in os.listdir(faces_dir):
    # Extract the name and id from the filename
    
    name_id = os.path.splitext(filename)[0] #extracting the name_id from file  

    # Load the face image
    img = face_recognition.load_image_file(os.path.join(faces_dir, filename))

    # Encode the face and append to the lists
    encoding = face_recognition.face_encodings(img)[0]
    known_face_encodings.append(encoding)
    known_face_names.append(name_id)
    count+=1
print(f'Total faces: {count}')    
print(known_face_names)

### Face Detection

In [ ]:
# Open the default camera device (0) for video capture
camera = cv2.VideoCapture(0)

# Loop over each frame from the camera
while True:
    # Read the frame from the camera
    ret, frame = camera.read()
    if not ret:
        print("Error reading frame from camera")
        break

    # Convert the frame to RGB (face_recognition library uses RGB)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and their encodings in the frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

### Encode Faces 

In [ ]:
# Initialize lists to store the recognized individual ids and their corresponding prediction scores
individual_ids = []
prediction_scores = []

# Loop over each face encoding in the current frame
for face_encoding in face_encodings:
    # Try to recognize the face
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
    name = "Unknown"

    # Find the index of the first matching face (if any)
    if True in matches:
        first_match_index = matches.index(True)
        name = known_face_names[first_match_index]
    
    # Add the individual id and prediction score to the lists
    individual_id = str(name_id.split("_")[1])
    individual_ids.append(individual_id)
    prediction_score = face_recognition.face_distance([known_face_encodings[first_match_index]], face_encoding)[0]
    prediction_scores.append(prediction_score)

### Recognize Images:

In [ ]:
# Initialize the recognized individuals dictionary
recognized_individuals = {}

# Loop over each individual id in the list of recognized ids
for individual_id in individual_ids:
    # If the id has not been recognized before, add to the dictionary with a count of 1
    if individual_id not in recognized_individuals:
        recognized_individuals[individual_id] = 1
    # If the id has been recognized before, increment the count
    else:
        recognized_individuals[individual_id] += 1

# Loop over each recognized individual in the dictionary
for individual_id, count in recognized_individuals.items():
    # If the individual has been recognized more than once, mark attendance
    if count > 1:
        name = "Name " + str(individual_id)
        # Get the current date and time
        date = datetime.datetime.now().strftime("%d-%m-%Y")
        time = datetime.datetime.now().strftime("%H:%M")
        # Write the attendance record to the CSV file
        with open("attendance.csv", "a+") as file:
            writer = csv.writer(file)
            writer.writerow([individual_id, name, date, time])

In [ ]:
import cv2
import face_recognition
import os
import numpy as np
import pandas as pd
import time
import datetime

# Load images from the given folder and return a list of face encodings and IDs
def load_images_from_folder(folder):
    encodings = []
    ids = []
    for filename in os.listdir(folder):
        img = face_recognition.load_image_file(os.path.join(folder, filename))
        encoding = face_recognition.face_encodings(img)[0]
        id = os.path.splitext(filename)[0]
        id = str(id)  # convert ID to string
        encodings.append(encoding)
        ids.append(id)
    return encodings, ids

# Train the face recognition model with the given encodings and IDs
def train(encodings, ids):
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(encodings, np.array(ids))
    return recognizer

# Recognize faces in the given frame using the given face recognition model
def recognize(frame, face_locations, face_encodings, recognizer, id_to_name):
    font = cv2.FONT_HERSHEY_SIMPLEX
    individual_names = []
    for encoding in face_encodings:
        individual_id, predict, _ = recognizer.predict(encoding)
        individual_id = str(individual_id)  # convert ID to string
        individual_name = id_to_name.get(individual_id, 'Unknown')
        individual_names.append(individual_name)
    for (top, right, bottom, left), name in zip(face_locations, individual_names):
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with the name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    return frame, individual_names

# Main function for recognizing faces in the live video from the webcam
def recognize_faces():
    # Load the face encodings and IDs from the "face dataset" directory
    encodings, ids = load_images_from_folder('face dataset')

    # Train the face recognition model with the encodings and IDs
    recognizer = train(encodings, ids)

    # Load the dictionary mapping IDs to names
    id_to_name = {
        'qd123': 'John Doe',
        'ab456': 'Jane Smith',
        # add more mappings as needed
    }

    # Initialize variables for attendance tracking
    col_names = ['Id', 'Name', 'Date', 'Time']
    attendance = pd.DataFrame(columns=col_names)

    # Open the default camera device (0) for video capture
    video_capture = cv2.VideoCapture(0)

    while True:
        # 

In [2]:
import cv2
import os
import numpy as np
import face_recognition
import pandas as pd
import datetime
import pickle
import time

# Path to the face dataset directory
faces_dir = 'face dataset/'

# Initialize arrays for the face encodings and corresponding names
known_face_encodings = []
known_face_names = []

# Loop over each file in the faces directory
for filename in os.listdir(faces_dir):
    # Load the image file
    image = face_recognition.load_image_file(os.path.join(faces_dir, filename))

    name_id = os.path.splitext(filename)[0] #extracting the name_id from file

    # Encode the face and append the encoding and name to the arrays
    face_encoding = face_recognition.face_encodings(image)[0]
    known_face_encodings.append(face_encoding)
    known_face_names.append(name_id)

# # Save the face encodings and names to a pickle file
# encodings_data = {'encodings': known_face_encodings, 'names': known_face_names}
# with open('encodings.pickle', 'wb') as f:
#     f.write(pickle.dumps(encodings_data))

# # Load the encodings from the pickle file
# with open('encodings.pickle', 'rb') as f:
#     encodings_data = pickle.load(f)
#     known_face_encodings = encodings_data['encodings']
#     known_face_names = encodings_data['names']

# Initialize the attendance dataframe
col_names = ['Name_ID', 'Date', 'Time']
attendance = pd.DataFrame(columns=col_names)

# Initialize the font for labeling the recognized faces
font = cv2.FONT_HERSHEY_SIMPLEX

# Open the default camera device (0) for video capture
camera = cv2.VideoCapture(0)

# Loop over each frame from the camera
while True:
    # Read the frame from the camera
    ret, frame = camera.read()
    if not ret:
        print("Error reading frame from camera")
        break

    # Convert the frame to RGB for face recognition
    rgb_frame = frame[:, :, ::-1]
    #rgb_frame = cv2.resize(image,(0,0),None,0.25,0.25)
    #rgb_frame = cv2.cvtColor(rgb_frame,cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop over each face in the frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Recognize the user ID for the face using the known encodings
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # Find the best match for the face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
            print(name)

            # Mark attendance for the recognized user
            ts = time.time()
            date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
            timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
            attendance.loc[len(attendance)] = [name_id, date, timeStamp]
            # Remove duplicates based on Id and Date columns
            attendance = attendance.drop_duplicates(subset=['Name_ID','Date'],keep='first')
            # Save attendance record to CSV file
            attendance.to_csv('Attendance.csv',index=False,header=True)
        else:
            name = "Unknown"
            print(name)

        # Draw a box and label around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 240, 0), 2)
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.75, (0, 240, 0), 2)

#     ## Record attendance
#         if name != "Unknown":
#             now = datetime.datetime.now()
#             date = now.strftime("%Y-%m-%d")
#             time = now.strftime("%H:%M:%S")
            
#             # Add the attendance to the dictionary
#             if name not in attendance:
#                 attendance[name] = []
#             attendance[name].append((date, time))
    
    # Display the resulting image
    cv2.imshow('Video', frame)
    
    # Quit if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    # Wait for a key press to exit the program
    k = cv2.waitKey(100) & 0xff 
    if k == 27: # Press 'ESC' for exiting video
        break
    elif (k == ord('c')):
        # Check if the directory exists and create it if it doesn't
        if not os.path.exists("CapturedImages"):
            os.mkdir("CapturedImages")
        # Save the captured image into the datasets folder
        cv2.imwrite(f'CapturedImages/{individual_name}_{time.strftime("%d%m%Y-%H%M")}.jpg', frame)
        #cv2.imwrite(r'C:\Users\Q209\Current projects\Face Recognition_jupyter/User_" + individual_name + ".jpg"',frame)
        break
# Do a bit of cleanup
print("\n [INFO] Exiting Program ")
# # Save the attendance to a CSV file
# with open('attendance.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(col_names)
    
#     for name, attendance_list in attendance.items():
#         for date, time in attendance_list:
#             writer.writerow([name, date, time])

# Release handle to the webcam
camera.release()
cv2.destroyAllWindows()

Harsh_QD138
y_QD0568
Unknown
Harsh_QD138
H_QD0354
H_QD0354
H_QD0354
Harsh_QD138
y_QD0568
y_QD0568
y_QD0568

 [INFO] Exiting Program 


In [2]:
import face_recognition
import cv2
import numpy as np
import os
import csv
import datetime
import pickle

# Load face encodings and names
encodings_file = "encodings.pickle"
data = pickle.loads(open(encodings_file, "rb").read())
known_face_encodings = data["encodings"]
known_face_names = data["names"]

# Initialize some variables
font = cv2.FONT_HERSHEY_SIMPLEX
attendance = {}
col_names = ['Name', 'Date', 'Time']

# Open the default camera device (0) for video capture
video_capture = cv2.VideoCapture(0)

while True:
    # Read the frame from the camera
    ret, frame = video_capture.read()
    
    # Convert the frame to RGB
    rgb_frame = frame[:, :, ::-1]
    
    # Find all the faces and face encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    
    # Loop through each face in this frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if this face is a match for any known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        
        # If we have a match, get the name from the known_face_names array
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
            
        # Draw a box around the face and label it with the name
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)
        
        # Record attendance
        if name != "Unknown":
            now = datetime.datetime.now()
            date = now.strftime("%d-%m-%Y")
            time = now.strftime("%H:%M")
            
            # Add the attendance to the dictionary
            if name not in attendance:
                attendance[name] = []
            attendance[name].append((date, time))
    
    # Display the resulting image
    cv2.imshow('Video', frame)
    
    # Quit if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the attendance to a CSV file
with open('attendance.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(col_names)
    
    for name, attendance_list in attendance.items():
        for date, time in attendance_list:
            writer.writerow([name, date, time])

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()